In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

In [28]:
#pip install unstructured
#!pip install yachalk
#!pip install pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
  Obtaining dependency information for jsonpickle>=1.4.1 from https://files.pythonhosted.org/packages/d3/25/6e0a450430b7aa194b0f515f64820fc619314faa289458b7dfca4a026114/jsonpickle-3.0.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.0 MB/s eta 0:00:00


In [2]:
##Setting Directory

In [3]:
## Input data directory
data_dir = "raw_input_data"
inputdirectory = Path(f"./resources/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/")
inputdirectory

PosixPath('resources/raw_input_data')

In [4]:
## Load Documents

In [7]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[0].page_content)


100%|█████████████████████████████████████████████| 1/1 [00:20<00:00, 20.61s/it]

Number of chunks =  3
At first, her government, under the administration of the _dogmatists_, was an absolute _despotism_. But, as the legislative continued to show traces of the ancient barbaric rule, her empire gradually broke up, and intestine wars introduced the reign of _anarchy;_ while the _sceptics_, like nomadic tribes, who hate a permanent habitation and settled mode of living, attacked from time to time those who had organized themselves into civil communities. But their number was, very happily, small; and thus they could not entirely put a stop to the exertions of those who persisted in raising new edifices, although on no settled or uniform plan. In recent times the hope dawned upon us of seeing those disputes settled, and the legitimacy of her claims established by a kind of _physiology_ of the human understanding—that of the celebrated Locke. But it was found that—although it was affirmed that this so-called queen could not refer her descent to any higher source than tha

In [12]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

/Users/dominikpichler/Documents/Pet_Projects/Athene
(3, 3)


,text,source,chunk_id
0,"At first, her government, under the administra...",resources/raw_input_data/The_critique_of_pure_...,816f22c183c546a29ec41dc3913383e0
1,"At present, as all methods, according to the g...",resources/raw_input_data/The_critique_of_pure_...,f1b5bb45622c42eca49db0994895e3bb
2,For it is in reality vain to profess _indiffer...,resources/raw_input_data/The_critique_of_pure_...,b3ac15d021c243a8a687a5f57f3e2844


In [13]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

In [14]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='mistral:latest')
    dfg1 = graph2Df(concepts_list)
    
    
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

    
    
    
    
    
dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

 [
   {
       "node\_1": "dogmatists",
       "node\_2": "administration",
       "edge": "The government of the queen was under the administration of the dogmatists."
   },
   {
       "node\_1": "despotism",
       "node\_2": "her government",
       "edge": "Her government was an absolute despotism."
   },
   {
       "node\_1": "legislative",
       "node\_2": "ancient barbaric rule",
       "edge": "The legislative continued to show traces of the ancient barbaric rule."
   },
   {
       "node\_1": "empire",
       "node\_2": "gradually broke up",
       "edge": "The empire gradually broke up."
   },
   {
       "node\_1": "anarchy",
       "node\_2": "reign of",
       "edge": "While the intestine wars introduced the reign of anarchy."
   },
   {
       "node\_1": "sceptics",
       "node\_2": "attacked",
       "edge": "But their number was small; and thus they could not entirely put a stop to the exertions of those who persisted in raising new edifices, although on no settled 

,node_1,node_2,edge,chunk_id,count
0,indifferentism,chaos and night,Indifferenism is the mother of chaos and night...,f1b5bb45622c42eca49db0994895e3bb,4
1,indifferentism,source,Indifferenism is the source of re-creation and...,f1b5bb45622c42eca49db0994895e3bb,4
2,indifferentism,prelude,Indifferenism is the prelude to the re-creatio...,f1b5bb45622c42eca49db0994895e3bb,4
3,indifference,object of inquiries,The object of inquiries cannot be indifferent ...,b3ac15d021c243a8a687a5f57f3e2844,4
4,indifference,pretended indifferentists,Pretended indifferentists unavoidably fall int...,b3ac15d021c243a8a687a5f57f3e2844,4


In [15]:
dfg1

,node_1,node_2,edge,chunk_id,count
0,indifferentism,chaos and night,Indifferenism is the mother of chaos and night...,f1b5bb45622c42eca49db0994895e3bb,4
1,indifferentism,source,Indifferenism is the source of re-creation and...,f1b5bb45622c42eca49db0994895e3bb,4
2,indifferentism,prelude,Indifferenism is the prelude to the re-creatio...,f1b5bb45622c42eca49db0994895e3bb,4
3,indifference,object of inquiries,The object of inquiries cannot be indifferent ...,b3ac15d021c243a8a687a5f57f3e2844,4
4,indifference,pretended indifferentists,Pretended indifferentists unavoidably fall int...,b3ac15d021c243a8a687a5f57f3e2844,4
5,indifference,matured judgement of the age,It is plainly not the effect of the levity but...,b3ac15d021c243a8a687a5f57f3e2844,4
6,indifference,illusory knowledge,"It is a call to reason, again to undertake the...",b3ac15d021c243a8a687a5f57f3e2844,4
7,critical investigation of pure reason,tribunal,This tribunal is nothing less than the Critica...,b3ac15d021c243a8a687a5f57f3e2844,4


In [16]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
32,object of inquiries,indifference,"b3ac15d021c243a8a687a5f57f3e2844,b3ac15d021c24...",4,contextual proximity
37,prelude,indifferentism,"f1b5bb45622c42eca49db0994895e3bb,f1b5bb45622c4...",3,contextual proximity
41,pretended indifferentists,indifference,"b3ac15d021c243a8a687a5f57f3e2844,b3ac15d021c24...",4,contextual proximity
46,source,indifferentism,"f1b5bb45622c42eca49db0994895e3bb,f1b5bb45622c4...",3,contextual proximity
50,tribunal,indifference,"b3ac15d021c243a8a687a5f57f3e2844,b3ac15d021c24...",4,contextual proximity


In [20]:
'''
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
'''
dfg = dfg1

In [1]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape
nodes

NameError: name 'pd' is not defined

In [22]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/2)

In [23]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  5
[['chaos and night'], ['critical investigation of pure reason', 'tribunal'], ['illusory knowledge', 'indifference', 'matured judgement of the age', 'pretended indifferentists'], ['indifferentism', 'prelude', 'source'], ['object of inquiries']]


In [24]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

100%|████████████████████████████████████████████| 1/1 [06:19<00:00, 379.46s/it]


,node,color,group
0,chaos and night,#57db94,1
1,critical investigation of pure reason,#b9db57,2
2,tribunal,#b9db57,2
3,illusory knowledge,#db5f57,3
4,indifference,#db5f57,3
5,matured judgement of the age,#db5f57,3
6,pretended indifferentists,#db5f57,3
7,indifferentism,#5784db,4
8,prelude,#5784db,4
9,source,#5784db,4


In [25]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [29]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html
